In [1]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

Dependency Injection

In [2]:
#Dependency injection means in programming, that there is a way for your code to declare things that it requires to work and use:

#And then that system will take care of doing whatever is needed to provide your code with those needed dependencies

In [3]:
from typing import Annotated
from fastapi import Depends, FastAPI

app = FastAPI()


async def common_parameters(q: str | None = None, skip: int = 0, limit: int = 100):
    return {"q":q, "skip": skip, "limit": limit}


@app.get('/items/')
async def read_items(commons: Annotated[dict, Depends(common_parameters)]):
    return commons

@app.get('/users/')
async def read_users(commons: Annotated[dict, Depends(common_parameters)]):
    return commons


uvicorn.run(app, port=8000)

INFO:     Started server process [25670]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:42956 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:42956 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25670]


In [3]:
#avoiding code duplication like above

from typing import Annotated
from fastapi import Depends, FastAPI

app = FastAPI()


async def common_parameters(q: str | None = None, skip: int = 0, limit: int = 100):
    return {"q": q, "skip": skip, "limit": limit}


CommonsDep = Annotated[dict, Depends(common_parameters)]


@app.get("/items/")
async def read_items(commons: CommonsDep):
    return commons


@app.get("/users/")
async def read_users(commons: CommonsDep):
    return commons



uvicorn.run(app, port=8000)


INFO:     Started server process [6497]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:54934 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54934 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54934 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:54934 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:57788 - "GET /items/?skip=0&limit=100 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [6497]


In [4]:
#To async or not to async

#As dependencies will also be called by fastapi, the same rules apply while defining your functions.


#You can use async def or normal def.


#You can declare dependencies with async def inside of normal def path operation functions, or def dependencies inside of async def path operation functions, etc.



from typing import Annotated
from fastapi import Depends, FastAPI


app = FastAPI()

fake_items_db = [{"item_name": "Foo"}, {"item_name": "Bar"}, {"item_name": "Baz"}]



class CommonQueryParams:
    def __init__(self, q: str | None = None, skip: int = 0, limit: int = 100):
        self.q = q
        self.skip = skip
        self.limit = limit


@app.get('/items/')
async def read_items(commons: Annotated[CommonQueryParams, Depends(CommonQueryParams)]):
    response = {}
    if commons.q:
        response.update({"q": commons.q})
    items = fake_items_db[commons.skip: commons.skip + commons.limit]
    response.update({'items': items})
    return response


uvicorn.run(app, port=8000)

INFO:     Started server process [6497]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:36892 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:36892 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:33438 - "GET /items/?skip=0&limit=100 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [6497]


Sub-dependencies

In [5]:
#You can create dependencies that have sub-dependencies.
#They can be as deep as you need them to be.
#FastAPI will take care of solving them.


#FIRST DEPENDENCY "DEPENDABLE"

from typing import Annotated
from fastapi import Cookie, Depends, FastAPI

app = FastAPI()


def query_extractor(q: str | None = None):
    return q


def query_or_cookie_extractor(
        q: Annotated[str, Depends(query_extractor)],
        last_query: Annotated[str | None, Cookie()] = None,
):
    if not q:
        return last_query
    return q



@app.get('/items/')
async def read_query(query_or_default: Annotated[str, Depends(query_or_cookie_extractor)]):
    return {'q_or_cookie': query_or_default}



uvicorn.run(app, port=8000)

INFO:     Started server process [6497]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:58786 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:58786 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:35076 - "GET /items/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [6497]


Using the same dependency multiple times

In [6]:
async def needy_dependency(fresh_value: Annotated[str, Depends(get_value, use_cache=False)]):
    return {"fresh_value": fresh_value}

NameError: name 'get_value' is not defined

Dependencies in path opeartion decorators

In [7]:
#In some cases you dont really need the return value of a dependency inside your path operation function.

#Or the dependency doesnt return a value.

#But you still need it to be executed/solved.

#For those cases, instead of declaring a path operation function parameter with Depends,you can add a list of dependencies to the path operation decorator.




Add dependencies to the path operation decorator

In [9]:
from typing import Annotated
from fastapi import Depends, FastAPI, Header, HTTPException


app = FastAPI()



async def verify_token(x_token: Annotated[str, Header()]):
    if x_token !="fake-super-secret-token":
        raise HTTPException(status_code=400, detail="X-Token header invalid")
    

async def verify_key(x_key: Annotated[str, Header()]):
    if x_key != "fake-super-secret-token":
        raise HTTPException(status_code=400, detail="X-key header invalid")



@app.get("/items/", dependencies=[Depends(verify_token), Depends(verify_key)])
async def read_items():
    return [{"item": "Foo"}, {"item": "Bar"}]


uvicorn.run(app, port=8000)

INFO:     Started server process [6497]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:48808 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:48808 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:44260 - "GET /items/ HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:46528 - "GET /items/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:50772 - "GET /items/ HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:42032 - "GET /items/ HTTP/1.1" 400 Bad Request


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [6497]


## Global Dependencies

In [10]:
#For some types of applications you might want to add dependencies to the whole application.

#Similar to the way you can add dependencies to the path operation decorators, you can add them to the FastAPI application.

#In that case, they will be applied to all the path operations in the application:



from fastapi import Depends, FastAPI, Header, HTTPException
from typing_extensions import Annotated


async def verify_token(x_token: Annotated[str, Header()]):
    if x_token != 'fake-super-secret-token':
        raise HTTPException(status_code=400, detail="X-Token header invalid")


async def verify_key(x_key: Annotated[str, Header()]):
    if x_key != 'fake-super-secret-key':
        raise HTTPException(status_code=400, detail="X-key header invalid")
    return x_key


app = FastAPI(dependencies=[Depends(verify_token), Depends(verify_key)])



@app.get('/items/')
async def read_items():
    return [{'item': "Portal Gun"}, {"item": "Plumbus"}]


@app.get('/users/')
async def read_users():
    return [{'username': "rick"}, {'username': 'Morty'}]



uvicorn.run(app, port=8000)


INFO:     Started server process [6497]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:41370 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:41370 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:40664 - "GET /items/ HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:53144 - "GET /items/ HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:57374 - "GET /items/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:43038 - "GET /users/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [6497]


## Dependencies with yield and try

In [ ]:
#If you use a try block in a dependency with yield, you will receive any exception that was thrown when using the dependency.

#So you can look for that specific exception inside the dependency with



#In the same way, you can use finally to make sure the exit steps are executed, no matter if there was an exception or not.

async def get_db():
    db = DBSession()
    try:
        yield db
    finally:
        db.close()




In [ ]:
# Sub-dependencies with yield
from typing import Annotated
from fastapi import Depends


async def depedency_a():
    dep_a = generate_dep_a()
    try:
        yield dep_a
    finally:
        dep_a.close()


async def dependency_b(dep_a: Annotated[DepA, Depends(depedency_a)]):
    dep_b = generate_dep_b()
    try:
        yield dep_b
    finally:
        dep_b.close(dep_a)

async def dependency_c(dep_b: Annotated[DepB, Depends(dependency_b)]):
    dep_c = generate_dep_c()
    try:
        yield dep_c
    finally:
        dep_c.close(dep_b)


        


Dependencies with yield and HTTPException